## Analytic 23 Code

#### OPIM5770 | Fall 2018 | Team 4

###### This notebook contains code to generate a report for analytic 23. Designed by Team 4.

In [1]:
# Import required modules
import pandas as pd
import numpy as np
import os

In [2]:
# Load the BSAK_BKPF (Note: with date parsing enabled, may take some time to complete)
parse_dates = [ 'Document_Date_in_Document'
               ,'Clearing_Date'
               ,'Day_On_Which_Accounting_Document_Was_Entered']

BSAK_BKPF_DF = pd.read_csv(r'./../../src/BSAK_BKPF.csv' 
                           , sep="|"
                           , quotechar="'"
                           , low_memory=False
                           , encoding='latin1'
                           , usecols = [ 
                                     'Company_Code',
                                     'Account_Number_of_Vendor_or_Creditor',
                                     'Reference_Document_Number',
                                     'Accounting_Document_Number',
                                     'Document_Type',
                                     'Number_of_Line_Item_Within_Accounting_Document',
                                     'Document_Number_of_the_Clearing_Document',
                                     'Reference_Document_Number',
                                     'Fiscal_Year',
                                     'Document_Date_in_Document',
                                     'Clearing_Date',
                                     'Day_On_Which_Accounting_Document_Was_Entered',
                                     'Debit_Credit_Indicator',
                                     'Currency_Key',
                                     'Amount_in_Document_Currency',
                                     'Amount_in_Local_Currency',
                                     'Reverse_Document_Number'
                                     ]
                           , dtype = { 
                                     'Company_Code': str,
                                     'Account_Number_of_Vendor_or_Creditor': str,
                                     'Reference_Document_Number': str,
                                     'Accounting_Document_Number': str,
                                     'Document_Type': str, 
                                     'Number_of_Line_Item_Within_Accounting_Document':int,
                                     'Document_Number_of_the_Clearing_Document': str,
                                     'Reference_Document_Number':str,
                                     'Fiscal_Year':int,
                                     'Document_Date_in_Document':str, # REQUIRES DATE PARSING
                                     'Clearing_Date': str, # REQUIRES DATE PARSING
                                     'Day_On_Which_Accounting_Document_Was_Entered': str, #REQUIRES DATE PARSING
                                     'Debit_Credit_Indicator': str,
                                     'Currency_Key':str,
                                     'Amount_in_Document_Currency': float,
                                     'Amount_in_Local_Currency':float,
                                     'Reverse_Document_Number':str
                                   }
                          , parse_dates=parse_dates)

BSAK_BKPF_DF.rename(columns=
          {
              'Company_Code': 'COMPANY_CODE',
              'Account_Number_of_Vendor_or_Creditor': 'VENDOR_ID',
              'Reference_Document_Number': 'VENDOR_INVOICE_NUMBER',
              'Accounting_Document_Number': 'ACCOUNTING_DOC_NUM',
              'Document_Type': 'DOCUMENT_TYPE',
              'Number_of_Line_Item_Within_Accounting_Document':'LINE_ITEM_ACCT_DOC_NUM',
              'Document_Number_of_the_Clearing_Document':'CLEARING_DOC_NUM',
              'Reference_Document_Number': 'REFERENCE_DOC_NBR',
              'Fiscal_Year': 'FISCAL_YEAR',
              'Document_Date_in_Document': 'DOCUMENT_DATE',
              'Clearing_Date': 'CLEARING_DATE',
              'Day_On_Which_Accounting_Document_Was_Entered': 'DAY_DOC_ENTERED',
              'Debit_Credit_Indicator': 'DR_CR_INDICATOR',
              'Currency_Key': 'DOC_CURRENCY_INDICATOR',
              'Amount_in_Document_Currency': 'INVOICE_                    AMT_DOC_CURRENCY',
              'Amount_in_Local_Currency': 'INVOICE_AMT_REPORT_CURRENCY',
              'Reverse_Document_Number': 'REVERSE_DOC_NBR',
          }, inplace=True)

In [3]:
# Filter out unnecessary data types
BSAK_BKPF_DF = BSAK_BKPF_DF[BSAK_BKPF_DF['DOCUMENT_TYPE'] != 'ZP']
BSAK_BKPF_DF = BSAK_BKPF_DF[BSAK_BKPF_DF['DOCUMENT_TYPE'] != 'KZ']

In [4]:
# Load the T001 dataframe
T001_DF = pd.read_csv(r'./../../src/T001.csv',
                     usecols = [ 'Company_Code', 
                                 'Name_of_Company_Code_or_Company',
                                 'Currency_Key'],
                     dtype = {  'Company Code': str,
                                 'Name_of_Company_Code_or_Company': str ,
                                 'Currency_Key': str
                              }
                     )

T001_DF = T001_DF.rename(columns= {
                    'Company_Code': 'COMPANY_CODE',
                    'Name_of_Company_Code_or_Company': 'COMPANY_NAME',
                    'Currency_Key': 'REPORT_CURRENCY_INDICATOR',
                    })

In [5]:
# Perform join operation
joinDF = pd.merge(BSAK_BKPF_DF,
                  T001_DF,
                 left_on='COMPANY_CODE',
                 right_on='COMPANY_CODE',
                 how='left')

In [6]:
# Add requested computed fields
joinDF['DAY_OF_WEEK'] = joinDF['DAY_DOC_ENTERED'].dt.weekday

##### GENERATE BENFORD REPORT BY VENDOR

In [7]:
# Get necessary fields and first digit for analysis
benfordDF = joinDF

benfordDF['REPORT_AMT_STRING'] = benfordDF['INVOICE_AMT_REPORT_CURRENCY'].apply(str)

benfordDF['REPORT_FIRST_DIGIT'] = joinDF['REPORT_AMT_STRING'].str[:1]

invoiceInputDF = benfordDF[['COMPANY_CODE',
                       'VENDOR_ID',
                       'COMPANY_NAME',
                       'DOCUMENT_TYPE',
                       'FISCAL_YEAR',
                       'DAY_DOC_ENTERED',
                       'DR_CR_INDICATOR',
                       'DOCUMENT_TYPE',
                       'CLEARING_DOC_NUM',
                       'REPORT_FIRST_DIGIT'
                      ]]

In [8]:
# Create subset for flagging
flagInputDF = invoiceInputDF[['VENDOR_ID',
                              'REPORT_FIRST_DIGIT']]

In [9]:
# Get first digit by vendor
flagDigitGroupBy = flagInputDF.groupby(['VENDOR_ID','REPORT_FIRST_DIGIT'], as_index=False).size().reset_index()
digitGroupByDF = flagDigitGroupBy.rename(columns={0:'VENDOR_FIRST_DIGIT_COUNT'})

In [10]:
# Calculate invoice total by vendor
flagTotalGroupBy = flagInputDF.groupby(['VENDOR_ID'], as_index=False).size().reset_index()
totalGroupByDF = flagTotalGroupBy.rename(columns={0:'VENDOR_TOTAL_COUNT'})

In [11]:
# Merge the count by digit to the total count
flagMerge = pd.merge(digitGroupByDF,
                  totalGroupByDF,
                 left_on='VENDOR_ID',
                 right_on='VENDOR_ID',
                 how='left')

In [12]:
# Calculate percentages by vendor
flagMerge['DIGIT_PERCENTAGE'] = (flagMerge['VENDOR_FIRST_DIGIT_COUNT'] / flagMerge['VENDOR_TOTAL_COUNT']) * 100

In [13]:
# Create Benford reference table
benfordRefDF = pd.DataFrame({ 'DIGIT' : ['1', '2', '3', '4', '5', '6', '7', '8', '9'],
                              'BENFORD_PROB': [30.1, 17.6, 12.5, 9.7, 7.9, 6.7, 5.8, 5.1, 4.6]                   
                            })

In [46]:
# Join the results
benfordFinalDF = pd.merge(flagMerge,
                          benfordRefDF,
                          left_on='REPORT_FIRST_DIGIT',
                          right_on='DIGIT',
                          how='left')

In [47]:
benfordFinalDF.head()

,VENDOR_ID,REPORT_FIRST_DIGIT,VENDOR_FIRST_DIGIT_COUNT,VENDOR_TOTAL_COUNT,DIGIT_PERCENTAGE,BENFORD_PROB,DIGIT
0,0000020660,1,1,6,16.666667,30.1,1
1,0000020660,2,2,6,33.333333,17.6,2
2,0000020660,3,3,6,50.000000,12.5,3
3,0001000198,1,16,68,23.529412,30.1,1
4,0001000198,2,3,68,4.411765,17.6,2


In [48]:
# Calculate the expected differences
benfordFinalDF['BENFORD_DIFFERENCE'] = abs(benfordFinalDF['DIGIT_PERCENTAGE'] - benfordFinalDF['BENFORD_PROB'])

In [49]:
benfordFinalDF

,VENDOR_ID,REPORT_FIRST_DIGIT,VENDOR_FIRST_DIGIT_COUNT,VENDOR_TOTAL_COUNT,DIGIT_PERCENTAGE,BENFORD_PROB,DIGIT,BENFORD_DIFFERENCE
0,0000020660,1,1,6,16.666667,30.1,1,13.433333
1,0000020660,2,2,6,33.333333,17.6,2,15.733333
2,0000020660,3,3,6,50.000000,12.5,3,37.500000
3,0001000198,1,16,68,23.529412,30.1,1,6.570588
4,0001000198,2,3,68,4.411765,17.6,2,13.188235
5,0001000198,3,8,68,11.764706,12.5,3,0.735294
6,0001000198,4,11,68,16.176471,9.7,4,6.476471
7,0001000198,5,8,68,11.764706,7.9,5,3.864706
8,0001000198,6,4,68,5.882353,6.7,6,0.817647
9,0001000198,7,5,68,7.352941,5.8,7,1.552941


In [50]:
# Get the total count
vendorCountDF = benfordFinalDF.groupby(['VENDOR_ID'], as_index=False).size().reset_index()
vendorCountDF = vendorCountDF.rename(columns={0:'VENDOR_TOTAL_COUNT'})

In [51]:
vendorCountDF

,VENDOR_ID,VENDOR_TOTAL_COUNT
0,0000020660,3
1,0001000198,9
2,0001003866,4
3,0001004877,9
4,0001005204,9
5,0001009072,2
6,0001009399,9
7,0001015480,9
8,0001015759,6
9,0001017141,4


In [52]:
# Get sum of Benford difference probability
totBenfordDiffDF = benfordFinalDF.groupby('VENDOR_ID').agg({'BENFORD_DIFFERENCE' : 'sum'}).reset_index()

In [65]:
# Perform needed merge
benfordFinalDF = pd.merge(left=vendorCountDF, right=totBenfordDiffDF, how='inner')

In [66]:
# Create overall risk groupings
benfordFinalDF['BENFORD_RISK'] = pd.cut(benfordFinalDF['BENFORD_DIFFERENCE'], [0,25,50,75,1000000],
                                         labels=['Low', 'Medium', 'High', 'Very High'])

benfordFinalDF['VOLUME_INDICATOR'] = pd.cut(benfordFinalDF['VENDOR_TOTAL_COUNT'], [0,2,4,6,1000000],
                                         labels=['Low', 'Medium', 'High', 'Very High'])

In [67]:
# Save detail copy of report
benfordFinalDF.to_csv(r'./../output/A23_ReportDtl.csv', index= False)

In [68]:
# Create summary version of the report
benfordFinalDF = benfordFinalDF[['VENDOR_ID','BENFORD_RISK','VOLUME_INDICATOR']]

In [69]:
# Export the summary version of the report
benfordFinalDF.to_csv(r'./../output/A23_ReportSummary.csv', index= False)